In [1]:
from google.colab import drive
from google.colab import files 
import os

drive.mount("/content/gdrive")
print(os.getcwd())
os.chdir("/content/gdrive/My Drive/AML2021")
print(os.getcwd())

Mounted at /content/gdrive
/content
/content/gdrive/.shortcut-targets-by-id/1_nr6od538RmnZRE1gB6vQYFsrHaUgS19/AML2021


In [2]:
import matplotlib.pyplot as plt
import numpy as np
import time

from tensorflow.keras import backend, optimizers, models, applications, metrics
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Input
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping, CSVLogger, TensorBoard
import tensorflow.keras.utils as utils

In [3]:
x_train = np.load("x_train.npy")
y_train = np.load("y_train.npy")

print(x_train.shape, y_train.shape)

(16324, 299, 299, 3) (16324, 2)


In [ ]:
# Adapt Constants
date = time.strftime("%d:%m:%Y, %H:%M:%S")
MODEL_NAME = f"ResNet101-{date}"
os.makedirs(f'models/{MODEL_NAME}', exist_ok = True) 
INPUT_SHAPE = (299, 299, 3)
NUM_CLASSES = 2
INITIAL_EPOCH = 0
NUM_EPOCHS = 120
OPTIMIZER = optimizers.SGD(learning_rate=0.1, momentum=0.9, nesterov=False)
BATCH_SIZE = 16

LRS = ReduceLROnPlateau(monitor='val_auroc', factor=0.1, patience=10, verbose=1, mode='auto', min_delta=0.0001)
ES = EarlyStopping(monitor='val_auroc', min_delta=0.0001, patience=15, verbose=0, mode='auto', baseline=None, restore_best_weights=False)


MC = ModelCheckpoint(f'models/{MODEL_NAME}/latest-{MODEL_NAME}.hdf5', monitor='val_auroc', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', save_freq="epoch")
MC_BEST = ModelCheckpoint(f'models/{MODEL_NAME}/best-{MODEL_NAME}.hdf5', monitor='val_auroc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', save_freq="epoch")
LOG = CSVLogger(f'models/{MODEL_NAME}/{MODEL_NAME}.log')
TB = TensorBoard(log_dir=f'models/{MODEL_NAME}/logs')

METRICS = [metrics.AUC(name='auroc', curve='ROC'),
           metrics.Accuracy(name='accuracy'),
           metrics.Precision(name='precision'),
           metrics.Recall(name='recall')
           ]

# Load data
assert(backend.image_data_format()=='channels_last')

# Model: preprocess, base model, top layer/classifier
i = Input(shape=INPUT_SHAPE)
x = applications.resnet.preprocess_input(i)

base_model = applications.ResNet101(include_top=False, weights="imagenet", input_shape=INPUT_SHAPE)
x = base_model(x)

x = GlobalAveragePooling2D(name='avg_pool')(x)
o = Dense(NUM_CLASSES, activation='softmax', name=f'fc{NUM_CLASSES}')(x)

model = models.Model(inputs=i, outputs=o)

# Train Model
if INITIAL_EPOCH != 0:
	model = models.load_model(f'models/{MODEL_NAME}/latest-{MODEL_NAME}.hdf5', compile=True)
else:
  model.compile(optimizer=OPTIMIZER, loss='categorical_crossentropy', metrics=METRICS)

model.fit(x=x_train, y=y_train, batch_size=BATCH_SIZE, validation_split = 0.2, callbacks = [MC, MC_BEST, LRS, ES, LOG, TB], epochs = NUM_EPOCHS, initial_epoch = INITIAL_EPOCH)

# Eval Model
x_test = np.load("x_test.npy")
y_test = np.load("y_test.npy")

model = models.load_model(f'models/{MODEL_NAME}/latest-{MODEL_NAME}.hdf5', compile=True)
out = model.evaluate(test_generator, batch_size=BATCH_SIZE, verbose=1)

# Save Results
with open("results.txt","a+") as f:
  f.write(f"{MODEL_NAME} test_loss: {out[0]}, test_auroc: {out[1]}, test_acc: {out[2]}, test_prec: {out[3]}, test_rec: {out[4]}, test_f1: {2* ((out[3]*out[4]) / (out[3]+out[4]))}\n")
print(f"wrote \"{MODEL_NAME} test_loss: {out[0]}, test_auroc: {out[1]}, test_acc: {out[2]}, test_prec: {out[3]}, test_rec: {out[4]}, test_f1: {2* ((out[3]*out[4]) / (out[3]+out[4]))}\n\" to results.txt ")

171458560/171446536 [==============================] - 2s 0us/step


/usr/local/lib/python3.7/dist-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


Epoch 1/120
817/817 [==============================] - 998s 1s/step - loss: 0.7629 - auroc: 0.6956 - accuracy: 0.0010 - precision: 0.6385 - recall: 0.6385 - val_loss: 0.5247 - val_auroc: 0.8324 - val_accuracy: 0.0000e+00 - val_precision: 0.7688 - val_recall: 0.7688

Epoch 00001: saving model to models/ResNet101-13:09:2021, 15:31:50/latest-ResNet101-13:09:2021, 15:31:50.hdf5

Epoch 00001: val_auroc improved from inf to 0.83243, saving model to models/ResNet101-13:09:2021, 15:31:50/best-ResNet101-13:09:2021, 15:31:50.hdf5
Epoch 2/120
817/817 [==============================] - 938s 1s/step - loss: 0.5281 - auroc: 0.8144 - accuracy: 0.0000e+00 - precision: 0.7352 - recall: 0.7352 - val_loss: 1.5112 - val_auroc: 0.6159 - val_accuracy: 0.0440 - val_precision: 0.6000 - val_recall: 0.6000

Epoch 00002: saving model to models/ResNet101-13:09:2021, 15:31:50/latest-ResNet101-13:09:2021, 15:31:50.hdf5

Epoch 00002: val_auroc improved from 0.83243 to 0.61588, saving model to models/ResNet101-13:09:

In [ ]:
#Plot ROC
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
y_pred = np.argmax(model.predict(x_test), axis=1)
y_true = np.argmax(y_test, axis=1)
fpr, tpr, thresholds = roc_curve(y_true, y_pred)
auc = auc(fpr, tpr)

plt.figure()
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(fpr, tpr, label='Keras (area = {:.3f})'.format(auc))
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve')
plt.legend(loc='best')
plt.show()